# Работаем с данными Преступления в Лос-Анджелесе

### Упражнение

1. Оптимизировать загрузку памяти датафреймом статистики преступлений в Лос-Анджелесе.
- Столбцы с дата должны быть представлены типом данных дат
2. Добавить столбец полного времени совершенного преступления (тип данных дата)


In [1]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import datetime

# задаем некоторые настройки pandas, регулирующие
# формат вывода
# pd.set_option('display.notebook_repr_html', False)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)

In [2]:
def str2date(n):
    d = n.split('/')
    return datetime.date(year=int(d[2]), month=int(d[0]), day=int(d[1]))

def str2time(n):
    if (len(n) == 4):
      return datetime.time(hour=int(n[0:2]), minute=int(n[2:4]))
    elif (len(n) == 1):
      return datetime.time(hour=int(n))  
    elif (len(n) == 2 and int(n) < 24):
      return datetime.time(hour=int(n))  
    elif (len(n) == 2 and int(n) > 23):
      return datetime.time(hour=0, minute=int(n))  
    else:
      return datetime.time(hour=int(n[0:1]), minute=int(n[1:3]))

# cчитываем данные и выводим первые 5 наблюдений
gl = pd.read_csv('Crime_Data_from_2010.csv', converters = {
    'Date Reported': str2date,
    'Date Occurred': str2date,
    'Time Occurred': str2time
})
gl.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,2013-03-14,2013-03-11,18:00:00,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,2010-01-25,2010-01-22,23:00:00,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,2013-03-19,2013-03-18,20:30:00,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,2010-11-11,2010-11-10,18:00:00,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,2014-01-11,2014-01-04,23:00:00,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [3]:
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()


,Date Reported,Date Occurred,Time Occurred,Area Name,Crime Code Description,MO Codes,Victim Sex,Victim Descent,Premise Description,Weapon Description,Status Code,Status Description,Address,Cross Street,Location
count,1692056,1692056,1692056,1692056,1691658,1508333,1536075,1536039,1688783,560476,1692054,1692056,1692056,281362,1692047
unique,2977,2977,1438,21,142,373145,5,20,306,79,9,6,71877,11522,61170
top,2017-08-07,2010-01-01,12:00:00,77th Street,BATTERY - SIMPLE ASSAULT,0344,M,H,STREET,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,6TH ST,BROADWAY,"(0, 0)"
freq,805,2155,93904,117760,154801,183280,788015,585667,377301,341251,1309316,1309316,3908,4942,6698


In [4]:
converted_obj = pd.DataFrame()

# пишем цикл, которой перебирает каждый столбец object, 
# проверяет его на соответствие заданному порогу 
# (количество уникальных значений должно быть меньше 50% 
# от общего количества значений), и если столбец 
# удовлетворяет порогу, преобразовывает его в тип category
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [5]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

# снова применяем функцию mem_usage, смотрим,
# сколько памяти занимают все столбцы типа object
# до и после преобразования в тип category
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

1481.87 MB
104.90 MB


,before,after
object,15.0,NaN
category,NaN,2.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [ ]:
new_col = []

for x in range(len(converted_obj)):
  new_col.append(datetime.datetime.combine(converted_obj.iloc[x]['Date Occurred'], converted_obj.iloc[x]['Time Occurred']))

converted_obj.info(memory_usage='deep')

In [ ]:
del gl